In [188]:
class Transition:
    def __init__(self, ins, neigh, outs, symm='none'):
        self.ins = ins
        self.outs = outs
        #NESW
        self.neigh = neigh
        self.symm = symm
        assert len(neigh)==4

    def to_str(self, outtransform, outstate):
        if (outstate[0]==self.symm):
            o=''
        else:
            o = f'symmetries:{self.symm}\n'
            outstate[0]=self.symm
        o = o+ str(self.ins)
        for c in self.neigh:
            o = o+ ", " + str(c)
        o = o+"; "+str(outtransform(self.outs))
        return o

In [189]:
groups = dict(
    gas = range(16),
    wall = range(16,32),
    outW = [i for i in range(16) if (i&1)!=0] + [i+16 for i in range(16) if (i&1)!=0],
    outN = [i for i in range(16) if (i&2)!=0] + [i+16 for i in range(16) if (i&2)!=0],
    outE = [i for i in range(16) if (i&4)!=0] + [i+16 for i in range(16) if (i&4)!=0],
    outS = [i for i in range(16) if (i&8)!=0] + [i+16 for i in range(16) if (i&8)!=0] + [34+i for i in range(8) if (i&2)!=0 ],

    # now last bit is active or not after 32
    # 32 -> standby live, 33 -> active live

    # Now the membrane
    # 34-41 -- 0b(is up?)(has ^)(is active?)
    # 42-57 -- imaginary states --  0b(ising energy)(is up?)(has ^)(is active?)
    active = [33, ] + [34+i for i in range(8) if (i&1)!=0 and (i&4)!=0 ],
    standby = [32, ] + [34+i for i in range(8) if (i&1)==0 and (i&4)!=0 ],
    ac_up_membrane =  [34+i for i in range(8) if (i&4)!=0 and (i%2)!=0],
    st_up_membrane =  [34+i for i in range(8) if (i&4)!=0 and (i%2)==0],
    ac_down_membrane =  [34+i for i in range(8) if (i&4)==0 and (i%2)!=0],
    st_down_membrane =  [34+i for i in range(8) if (i&4)==0 and (i%2)==0],
)
print(groups)

{'gas': range(0, 16), 'wall': range(16, 32), 'outW': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31], 'outN': [2, 3, 6, 7, 10, 11, 14, 15, 18, 19, 22, 23, 26, 27, 30, 31], 'outE': [4, 5, 6, 7, 12, 13, 14, 15, 20, 21, 22, 23, 28, 29, 30, 31], 'outS': [8, 9, 10, 11, 12, 13, 14, 15, 24, 25, 26, 27, 28, 29, 30, 31, 36, 37, 40, 41], 'active': [33, 39, 41], 'standby': [32, 38, 40], 'ac_up_membrane': [39, 41], 'st_up_membrane': [38, 40], 'ac_down_membrane': [35, 37], 'st_down_membrane': [34, 36]}


In [190]:
trans=[]

for i in range(16):
    n = (f'{"any-" if ((i&8)==0) else ""}outS',
         f'{"any-" if ((i&1)==0) else ""}outW',
         f'{"any-" if ((i&2)==0) else ""}outN',
         f'{"any-" if ((i&4)==0) else ""}outE'
    )
    trans.append(Transition('gas' if i>0 else 'gas-(0)',n,i))

for i in range(16,32):
    n = (f'{"any-" if ((i&8)==0) else ""}outS',
         f'{"any-" if ((i&1)==0) else ""}outW',
         f'{"any-" if ((i&2)==0) else ""}outN',
         f'{"any-" if ((i&4)==0) else ""}outE'
    )
    trans.append(Transition('wall',n,i))

trans.append(Transition(33, ('any-standby','any-standby','standby','standby'), 0, 'permute'))
trans.append(Transition(0, ('any-standby','any-standby','standby','standby'), 33, 'permute'))
trans.append(Transition(32, ('any','any','any','any'), 32, 'permute'))
trans.append(Transition(33, ('any','any','any','any'), 33, 'permute'))


for i in range(16):
    n = ('standby' if ((i&4==0) != (i&8==0)) else 'any-standby',
         'any',
         f'{"any-" if ((i&2)==0) else ""}outN',
         'any',
    )
    ins = 'up_membrane' if (i&4)!=0 else 'down_membrane'
    ins = ('ac_'+ins) if (i&1)!=0 else ('st_'+ins)
    trans.append(Transition(ins, n, 42+i))




In [191]:

perm = [[5, 10], [32,33]]
#adding wall reflection
# there will be some duplicates, but shouldn't matter
for i in range(16):
    b = list(bin(i)[2:].zfill(4))
    b[0],b[2] = b[2],b[0]
    b[1],b[3] = b[3],b[1]
    perm.append([16+i,16+int(''.join(b),2)])

#membrane loop
perm.append([42+i for i in [0b0011,0b1100,0b1101,0b0010]])
perm.append([42+i for i in [0b0111,0b1000,0b1001,0b0110]])


#membrane alternation
for i in range(16):
    if (((i&8)==0) == ((i&2)==0)) and ((i&1)==0):
        perm.append([42+i, 42 + (i^1)])


def doperm(s):
    global perm
    for chain in perm:
        if s in chain:
            return chain[(chain.index(s)+1)%len(chain)]
    return s

def realize(s):
    if 42<=s<=57:
        return ((s-42)&7) + 34
    else:
        return s
print(perm)

[[5, 10], [32, 33], [16, 16], [17, 20], [18, 24], [19, 28], [20, 17], [21, 21], [22, 25], [23, 29], [24, 18], [25, 22], [26, 26], [27, 30], [28, 19], [29, 23], [30, 27], [31, 31], [45, 54, 55, 44], [49, 50, 51, 48], [42, 43], [46, 47], [52, 53], [56, 57]]


In [192]:
outpstr = ''
for n,g in groups.items():
    outpstr += f'{n} = {tuple(g)}\n'
os = ['None', ]
for t in trans:
    outpstr += t.to_str(lambda s: realize(doperm(s)),os) + '\n'

print(outpstr)

gas = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
wall = (16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31)
outW = (1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31)
outN = (2, 3, 6, 7, 10, 11, 14, 15, 18, 19, 22, 23, 26, 27, 30, 31)
outE = (4, 5, 6, 7, 12, 13, 14, 15, 20, 21, 22, 23, 28, 29, 30, 31)
outS = (8, 9, 10, 11, 12, 13, 14, 15, 24, 25, 26, 27, 28, 29, 30, 31, 36, 37, 40, 41)
active = (33, 39, 41)
standby = (32, 38, 40)
ac_up_membrane = (39, 41)
st_up_membrane = (38, 40)
ac_down_membrane = (35, 37)
st_down_membrane = (34, 36)
symmetries:none
gas-(0), any-outS, any-outW, any-outN, any-outE; 0
gas, any-outS, outW, any-outN, any-outE; 1
gas, any-outS, any-outW, outN, any-outE; 2
gas, any-outS, outW, outN, any-outE; 3
gas, any-outS, any-outW, any-outN, outE; 4
gas, any-outS, outW, any-outN, outE; 10
gas, any-outS, any-outW, outN, outE; 6
gas, any-outS, outW, outN, outE; 7
gas, outS, any-outW, any-outN, any-outE; 8
gas, outS, outW, any-outN, any-ou

In [193]:
#Icons
stateicons = {
1 : '31.$31.$31.$31.$31.$31.$31.$31.$31.$31.$5.A25.$4.2A25.$3.3A25.$2.4A25.$.26A4.$27A4.$.26A4.$2.4A25.$3.3A25.$4.2A25.$5.A25.$31.$31.$31.$31.$31.$31.$31.$31.$31.$31.!',
2 : '15.A15.$14.3A14.$13.5A13.$12.7A12.$11.9A11.$10.11A10.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$31.$31.$31.$31.!',
3 : '15.A15.$14.3A14.$13.5A13.$12.7A12.$11.9A11.$10.11A10.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$5.A8.3A14.$4.2A8.3A14.$3.3A8.3A14.$2.4A8.3A14.$.26A4.$27A4.$.26A4.$2.4A8.3A14.$3.3A8.3A14.$4.2A8.3A14.$5.A8.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$31.$31.$31.$31.!',
4 : '31.$31.$31.$31.$31.$31.$31.$31.$31.$31.$25.A5.$25.2A4.$25.3A3.$25.4A2.$4.26A.$4.27A$4.26A.$25.4A2.$25.3A3.$25.2A4.$25.A5.$31.$31.$31.$31.$31.$31.$31.$31.$31.$31.!',
5 : '31.$31.$31.$31.$31.$31.$31.$31.$31.$31.$5.A19.A5.$4.2A19.2A4.$3.3A19.3A3.$2.4A19.4A2.$.29A.$31A$.29A.$2.4A19.4A2.$3.3A19.3A3.$4.2A19.2A4.$5.A19.A5.$31.$31.$31.$31.$31.$31.$31.$31.$31.$31.!',
6 : '15.A15.$14.3A14.$13.5A13.$12.7A12.$11.9A11.$10.11A10.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A8.A5.$14.3A8.2A4.$14.3A8.3A3.$14.3A8.4A2.$4.26A.$4.27A$4.26A.$14.3A8.4A2.$14.3A8.3A3.$14.3A8.2A4.$14.3A8.A5.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$31.$31.$31.$31.!',
7 : '15.A15.$14.3A14.$13.5A13.$12.7A12.$11.9A11.$10.11A10.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$5.A8.3A8.A5.$4.2A8.3A8.2A4.$3.3A8.3A8.3A3.$2.4A8.3A8.4A2.$.29A.$31A$.29A.$2.4A8.3A8.4A2.$3.3A8.3A8.3A3.$4.2A8.3A8.2A4.$5.A8.3A8.A5.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$31.$31.$31.$31.!',
8 : '31.$31.$31.$31.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$10.11A10.$11.9A11.$12.7A12.$13.5A13.$14.3A14.$15.A15.!',
9 : '31.$31.$31.$31.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$5.A8.3A14.$4.2A8.3A14.$3.3A8.3A14.$2.4A8.3A14.$.26A4.$27A4.$.26A4.$2.4A8.3A14.$3.3A8.3A14.$4.2A8.3A14.$5.A8.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$10.11A10.$11.9A11.$12.7A12.$13.5A13.$14.3A14.$15.A15.!',
10 : '15.A15.$14.3A14.$13.5A13.$12.7A12.$11.9A11.$10.11A10.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$10.11A10.$11.9A11.$12.7A12.$13.5A13.$14.3A14.$15.A15.!',
11 : '15.A15.$14.3A14.$13.5A13.$12.7A12.$11.9A11.$10.11A10.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$5.A8.3A14.$4.2A8.3A14.$3.3A8.3A14.$2.4A8.3A14.$.26A4.$27A4.$.26A4.$2.4A8.3A14.$3.3A8.3A14.$4.2A8.3A14.$5.A8.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$10.11A10.$11.9A11.$12.7A12.$13.5A13.$14.3A14.$15.A15.!',
12 : '31.$31.$31.$31.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A8.A5.$14.3A8.2A4.$14.3A8.3A3.$14.3A8.4A2.$4.26A.$4.27A$4.26A.$14.3A8.4A2.$14.3A8.3A3.$14.3A8.2A4.$14.3A8.A5.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$10.11A10.$11.9A11.$12.7A12.$13.5A13.$14.3A14.$15.A15.!',
13 : '31.$31.$31.$31.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$5.A8.3A8.A5.$4.2A8.3A8.2A4.$3.3A8.3A8.3A3.$2.4A8.3A8.4A2.$.29A.$31A$.29A.$2.4A8.3A8.4A2.$3.3A8.3A8.3A3.$4.2A8.3A8.2A4.$5.A8.3A8.A5.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$10.11A10.$11.9A11.$12.7A12.$13.5A13.$14.3A14.$15.A15.!',
14 : '15.A15.$14.3A14.$13.5A13.$12.7A12.$11.9A11.$10.11A10.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$14.3A8.A5.$14.3A8.2A4.$14.3A8.3A3.$14.3A8.4A2.$4.26A.$4.27A$4.26A.$14.3A8.4A2.$14.3A8.3A3.$14.3A8.2A4.$14.3A8.A5.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$10.11A10.$11.9A11.$12.7A12.$13.5A13.$14.3A14.$15.A15.!',
15 : '15.A15.$14.3A14.$13.5A13.$12.7A12.$11.9A11.$10.11A10.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$5.A8.3A8.A5.$4.2A8.3A8.2A4.$3.3A8.3A8.3A3.$2.4A8.3A8.4A2.$.29A.$31A$.29A.$2.4A8.3A8.4A2.$3.3A8.3A8.3A3.$4.2A8.3A8.2A4.$5.A8.3A8.A5.$14.3A14.$14.3A14.$14.3A14.$14.3A14.$10.11A10.$11.9A11.$12.7A12.$13.5A13.$14.3A14.$15.A15.!',
16 : '''31B$31B$31B$31B$31B$31B$31B$31B$31B$31B$31B$31B$31B$31B$31B$31B$31B$
31B$31B$31B$31B$31B$31B$31B$31B$31B$31B$31B$31B$31B$31B!''',
**dict.fromkeys(range(17,32),'''
31C$31C$31C$31C$31C$31C$31C$31C$31C$31C$31C$31C$31C$31C$31C$31C$31C$
31C$31C$31C$31C$31C$31C$31C$31C$31C$31C$31C$31C$31C$31C!'''),

**dict.fromkeys([32, 38],'''
31D$31D$31D$31D$31D$31D$31D$31D$31D$31D$31D$31D$31D$31D$31D$31D$31D$
31D$31D$31D$31D$31D$31D$31D$31D$31D$31D$31D$31D$31D$31D!'''),

**dict.fromkeys([33, 39],'''
31E$31E$31E$31E$31E$31E$31E$31E$31E$31E$31E$31E$31E$31E$31E$31E$31E$
31E$31E$31E$31E$31E$31E$31E$31E$31E$31E$31E$31E$31E$31E!'''),

34 : '''
7$7.16D$7.16D$7.16D$7.3D10.3D$7.3D10.3D$7.3D10.3D$7.3D10.3D$7.3D10.3D
$7.3D10.3D$7.3D10.3D$7.3D10.3D$7.3D10.3D$7.3D10.3D$7.16D$7.16D$7.16D!''',

35 : '''
7$7.16E$7.16E$7.16E$7.3E10.3E$7.3E10.3E$7.3E10.3E$7.3E10.3E$7.3E10.3E
$7.3E10.3E$7.3E10.3E$7.3E10.3E$7.3E10.3E$7.3E10.3E$7.16E$7.16E$7.16E!''',

36 : '''
7$7.16D$7.16D$7.16D$7.3D10.3D$7.3D4.2A4.3D$7.3D4.2A4.3D$7.3D3.4A3.3D$
7.3D3.4A3.3D$7.3D3.4A3.3D$7.3D3.4A3.3D$7.3D4.2A4.3D$7.3D4.2A4.3D$7.3D
10.3D$7.16D$7.16D$7.16D!''',

37 : '''
7$7.16E$7.16E$7.16E$7.3E10.3E$7.3E10.3E$7.3E4.2A4.3E$7.3E4.2A4.3E$7.
3E3.4A3.3E$7.3E3.4A3.3E$7.3E4.2A4.3E$7.3E4.2A4.3E$7.3E10.3E$7.3E10.3E
$7.16E$7.16E$7.16E!''',

40 : '''
31D$31D$31D$31D$31D$31D$31D$31D$31D$31D$31D$11D9.11D$11D3.3A3.11D$11D
3.3A3.11D$11D2.5A2.11D$11D2.5A2.11D$11D2.5A2.11D$11D3.3A3.11D$11D3.3A
3.11D$11D9.11D$31D$31D$31D$31D$31D$31D$31D$31D$31D$31D$31D!''',

41 : '''
31E$31E$31E$31E$31E$31E$31E$31E$31E$31E$31E$11E9.11E$11E3.3A3.11E$11E
3.3A3.11E$11E2.5A2.11E$11E2.5A2.11E$11E2.5A2.11E$11E3.3A3.11E$11E3.3A
3.11E$11E9.11E$31E$31E$31E$31E$31E$31E$31E$31E$31E$31E$31E!''',
}

colmap = dict(
A = 0xEEEEEF,
B = 0x446611,
C = 0xAAAA33,
D = 0x550000,
E = 0xFF1111,
)

statecolors = dict()
for s,ico in stateicons.items():
    ncol= 0
    n = 0
    colsum = [0,0,0]
    for c in ico.replace('\n','')[:-1]:
        if c in ('$','.') :
            n=0
        elif c in colmap:
            ncol+=max(1,n)
            colsum[0] += max(1,n)*(colmap[c]&0xFF0000)/(0x10000*31*31*256)
            colsum[1] += max(1,n)*(colmap[c]&0x00FF00)/(0x100  *31*31*256)
            colsum[2] += max(1,n)*(colmap[c]&0x0000FF)/(0x1    *31*31*256)
        else:
            n*=10
            n+=int(c)
    norm = max(colsum[0],colsum[1],colsum[2])
    if norm<=0.5:
        f = pow(norm,0.5)
    else:
        f = 1
    statecolors[s] = (
        colsum[0]/f,
        colsum[1]/f,
        colsum[2]/f,
    )
    print(s,ncol,norm)

colorsstr = ''
for s,c in sorted(statecolors.items()):
    colorsstr += f'{int(255*c[0])} {int(255*c[1])} {int(255*c[2])} : {s}\n'

icosstr = ''
for s,ic in stateicons.items():
    icosstr += f'#C {s}\nx=31, y=31, rule=//10\n{ic.strip()}\n\n'


1 99 0.09617667143600414
2 99 0.09617667143600409
3 189 0.18361000910509892
4 99 0.09617667143600414
5 129 0.12532111732570236
6 189 0.18361000910509892
7 219 0.21275445499479717
8 99 0.09617667143600413
9 189 0.18361000910509898
10 129 0.12532111732570234
11 219 0.21275445499479717
12 189 0.18361000910509898
13 219 0.21275445499479725
14 219 0.21275445499479717
15 249 0.24189890088449542
16 961 0.3984375000000002
17 961 0.6640624999999996
18 961 0.6640624999999996
19 961 0.6640624999999996
20 961 0.6640624999999996
21 961 0.6640624999999996
22 961 0.6640624999999996
23 961 0.6640624999999996
24 961 0.6640624999999996
25 961 0.6640624999999996
26 961 0.6640624999999996
27 961 0.6640624999999996
28 961 0.6640624999999996
29 961 0.6640624999999996
30 961 0.6640624999999996
31 961 0.6640624999999996
32 961 0.3320312499999998
38 961 0.3320312499999998
33 961 0.9960937499999994
39 961 0.9960937499999994
34 156 0.053898933402705546
35 156 0.1616968002081165
36 180 0.077116935483871
37 172 0.

In [194]:
with open('ising_temp_template.ruel','r') as fh:
    finalfile = (fh.read()
        .replace('[[[PASTE TRANSITIONS HERE]]]', outpstr)
        .replace('[[[PASTE ICONS HERE]]]', icosstr)
        .replace('[[[PASTE COLORS HERE]]]', colorsstr)
    )

with open('ising_temp.ruel','w') as fh:
    fh.write(finalfile)